<a href="https://colab.research.google.com/github/DNAS-bioDataSci/TREDNet_v1.1-colab/blob/main/TREDNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TREDNet for Google Colab
September 23, 2025

## Quick Start

* Requirements:
  * Minimum: T4 GPU based runtime node in Google Collab

* Run the cells in this notebook to:
  * Git the latest TREDNet python code for Google Colab
  * Fetch genomic data from a remote university server
  * Install Conda and Tensorflow on a GPU enabled node
  * Run TREDNet Deep Learning on the input files
  * Show TREDNet's sumary performance metrics

* If you have any config issues, see the **System Validation** section to help troubleshoot.

## Credits
This TREDNet-colab project was forked from [TREDNet v1.1 code](https://github.com/tanoManzo/TREDNet_v1.1) and ported to Google Colab for NIH by Jeremy Chambers @ [BioDataSci](https://biodatasci.com). Contact: [jeremy@bioDataSci.com](email:jeremy@bioDataSci.com)

# What TREDNet code is for
GWAS → fine-map → TREDNet → assays → translation
- GWAS signals identify disease-associated loci (e.g., T2D, glycemic traits).
- Fine-mapping narrows these down to credible sets of variants.
- TREDNet scores variants using enhancer damage/perturbation metrics.
- Functional validation (CRISPR, reporter assays, TF binding) tests high-priority candidates.
- Translational medicine leverages validated findings to inform biomarkers, drug targets, or therapeutic strategies.

TREDNet is a genomics tool developed at NIH for functional studies with translational potential. It is particularly powerful for diseases where noncoding enhancer variants drive pathology, like type 2 diabetes (T2D).

TREDNet implements a two-stage deep convolutional neural network that, starting from raw DNA sequence, predicts epigenomic features and then enhancer activity, and uses “saturated mutagenesis” style perturbations to prioritize which noncoding variants are likely to affect enhancer function in pancreatic islets.

It demonstrates that modeling pancreatic islet enhancers using deep learning identifies candidate causal variants at loci associated with T2D and glycemic traits.

See paper #1 in table below for details.

## What this Fork contributes
* Simplifies the `tf2.15-gpu.yml` packages so that the python package manager could solve the installation for the Conda, Cuda, Tensorflow, Python and other package version constraints.
* Adds this Colab notebook to document interactive configuration of a GPU node with the conda/tensorflow environment.
* Captures the runtime outputs end-to-end so that a future user knows "what good looks like" and where the "gotchas" are.
* Captures System Validation notes for a T4 GPU node in Google Colab.

Original code for TREDNet is found here: https://github.com/tanoManzo/TREDNet_v1.1 and is based on the papers below.

## Papers:
| # | Reference (title linked) | Alt/Indexing Link |
|---|:--------------------------|:------------------|
| 1 | [Hudaiberdiev S, et al. “Modeling islet enhancers using deep learning identifies candidate causal variants at loci associated with T2D and glycemic traits.” PNAS (2023)](https://www.pnas.org/doi/10.1073/pnas.2206612120) | [PubMed](https://pubmed.ncbi.nlm.nih.gov/37603758) |
| 2 | Davis CA, et al. “The ENCODE: data portal update.” Nucleic Acids Research (2018) | [Pubmed](https://pubmed.ncbi.nlm.nih.gov/29126249) |
| 3 | [Roadmap Epigenomics Consortium, et al. “Integrative analysis of 111 reference human epigenomes.” *Nature* (2015)](https://doi.org/10.1038/nature14248) | [Nature](https://doi.org/10.1038/nature14248) / [PubMed Central](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4530010/) |
| 4 | [Li S, Hannenhalli S, Ovcharenko I. “De novo human brain enhancers created by single-nucleotide mutations.” *Science Advances* (2023)](https://www.science.org/doi/10.1126/sciadv.add2911) | [Science Advances](https://www.science.org/doi/10.1126/sciadv.add2911) / [PubMed](https://pubmed.ncbi.nlm.nih.gov/38978133/) |
| 5 | [Huang D, Ovcharenko I. “The contribution of silencer variants to human diseases.” *Genome Biology* (2024)](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-024-03328-1) | [Genome Biology](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-024-03328-1) / [PubMed](https://pubmed.ncbi.nlm.nih.gov/38978133/) / [PMC](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC11232194/) |
| 6 | [Avsec Ž, et al. “Effective gene expression prediction from sequence by integrating long-range interactions.” *Nature Methods* (2021)](https://www.nature.com/articles/s41592-021-01252-x) | [Nature Methods](https://www.nature.com/articles/s41592-021-01252-x) / [PubMed](https://pubmed.ncbi.nlm.nih.gov/34608324/) / [PMC](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8490152/) |
| 7 | [NVIDIA. “Hopper Architecture In-Depth.” NVIDIA Technical Blog (2022)](https://developer.nvidia.com/blog/nvidia-hopper-architecture-in-depth/) | [NVIDIA Blog](https://developer.nvidia.com/blog/nvidia-hopper-architecture-in-depth/) / (PDF mirror: *if available, insert link*) |

Also documented at: [BioDataSci Wiki :: TREDNet Mockup in Google Collab](https://biodatasci.atlassian.net/wiki/spaces/BCC/pages/2112126980/TREDNet+Mockup+in+Google+Colab)

In [1]:
# Check if you're connected to a GPU (if not, select one in Google Collab!)
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0 or gpu_info.find('not found') >= 0:
  print("Not connected to a GPU.")
  print("Go to 'Runtime → Change runtime type → Hardware accelerator → T4 GPU' in Colab.")
else:
  print(gpu_info)

Thu Sep 25 14:15:46 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   33C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Git Clone
Note that the repo contains very large files that are transparently handled by LFS as long as there's enough remote quota left (Github Enterprise account is required remotely).  

## Workaround in case of Error cloning .h5 files
If issuing the next `git clone` below experiences `Error dowloading v0.1/phase_one_model.h5`, this is due to a remote LFS quota block outside of our control.

Workaround: download the files below manually from the Github repo, path `TREDNet_v1.1-colab/v0.1` (i.e., use RAW download in Github console):
* phase_one_model.h5
* phase_one_weights.h5
* phase_two_mdoel.hdf5

... and put them in your Colab GPU node's ephemeral storage at `/content/TREDNet_v1.1-colab/v0.1`

In [2]:
# Setup git LFS ... you only have to do this once per repo per machine.
!git lfs install

Git LFS initialized.


In [3]:
# Clone your GitHub repository
!git clone https://github.com/DNAS-bioDataSci/TREDNet_v1.1-colab

Cloning into 'TREDNet_v1.1-colab'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 91 (delta 40), reused 37 (delta 7), pack-reused 0 (from 0)
Receiving objects: 100% (91/91), 166.70 KiB | 12.82 MiB/s, done.
Resolving deltas: 100% (40/40), done.
Error downloading object: v0.1/phase_one_model.h5 (fc4b446): Smudge error: Error downloading v0.1/phase_one_model.h5 (fc4b4461e813418ea5444dab66bb088efb3ab92a208f1003a253562a72650f2b): batch response: This repository exceeded its LFS budget. The account responsible for the budget should increase it to restore access.

Errors logged to /content/TREDNet_v1.1-colab/.git/lfs/logs/20250925T141617.555595365.log
Use `git lfs logs last` to view the log.
error: external filter 'git-lfs filter-process' failed
fatal: v0.1/phase_one_model.h5: smudge filter lfs failed
You can inspect what was checked out with 'git status'
and retry with 'git restore --source

All the files EXCEPT the LFS tracked files were installed into the Colab GPU host due to the LFS quota-limit mentioned earlier.  So we have to manually download the LFS tracked files from the Github GUI and then upload them to the Colab GPU host...

In [ ]:
# Here's what Git LFS is tracking:
%cd /content/TREDNet_v1.1-colab
!git lfs track

/content/TREDNet_v1.1-colab
Listing tracked patterns
    v0.1/phase_two_model.hdf5 (.gitattributes)
    v0.1/phase_one_model.h5 (.gitattributes)
    v0.1/phase_one_weights.h5 (.gitattributes)
Listing excluded patterns


In [ ]:
# show git log -- use workaround above for LFS tracked files that error out.
!git lfs logs last

git-lfs/3.0.2 (GitHub; linux amd64; go 1.18.1)
git version 2.34.1

$ git-lfs filter-process
Error downloading object: v0.1/phase_one_model.h5 (fc4b446): Smudge error: Error downloading v0.1/phase_one_model.h5 (fc4b4461e813418ea5444dab66bb088efb3ab92a208f1003a253562a72650f2b): batch response: This repository exceeded its LFS budget. The account responsible for the budget should increase it to restore access.

This repository exceeded its LFS budget. The account responsible for the budget should increase it to restore access.
batch response
github.com/git-lfs/git-lfs/errors.newWrappedError
	github.com/git-lfs/git-lfs/errors/types.go:225
github.com/git-lfs/git-lfs/errors.Wrap
	github.com/git-lfs/git-lfs/errors/errors.go:74
github.com/git-lfs/git-lfs/tq.(*tqClient).Batch
	github.com/git-lfs/git-lfs/tq/api.go:95
github.com/git-lfs/git-lfs/tq.Batch
	github.com/git-lfs/git-lfs/tq/api.go:43
github.com/git-lfs/git-lfs/tq.(*TransferQueue).enqueueAndCollectRetriesFor
	github.com/git-lfs/git-lfs/t

In [ ]:
!git config --global user.name "jeremy@bioDataSci.com"
!git config --global user.email "jeremy@bioDataSci.com"

In [ ]:
!ls -alF

total 84
drwxr-xr-x 6 root root  4096 Sep 23 14:54 ./
drwxr-xr-x 1 root root  4096 Sep 23 14:54 ../
-rw-r--r-- 1 root root 12149 Sep 23 14:53 back_run_two_phase.py
drwxr-xr-x 2 root root  4096 Sep 23 14:55 fasta/
drwxr-xr-x 9 root root  4096 Sep 23 14:53 .git/
-rw-r--r-- 1 root root   184 Sep 23 14:53 .gitattributes
-rw-r--r-- 1 root root    20 Sep 23 14:53 .gitignore
drwxr-xr-x 3 root root  4096 Sep 23 14:53 input_training_trednet/
-rw-r--r-- 1 root root    13 Sep 23 14:53 list_of_biosamples_testing.txt
-rw-r--r-- 1 root root   279 Sep 23 14:53 README.md
-rwxr-xr-x 1 root root  1518 Sep 23 14:53 submit_testing.sh*
-rw-r--r-- 1 root root   275 Sep 23 14:53 test.py
-rw-r--r-- 1 root root  1498 Sep 23 14:53 tf2.15-gpu.yml
-rw-r--r-- 1 root root 20307 Sep 23 14:53 TREDNet.ipynb
drwxr-xr-x 2 root root  4096 Sep 23 14:53 v0.1/


In [ ]:
# get human genome data
!mkdir /content/TREDNet_v1.1-colab/fasta
!rsync -avzP rsync://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/latest/hg38.fa.gz /content/TREDNet_v1.1-colab/fasta
!gunzip /content/TREDNet_v1.1-colab/fasta/hg38.fa.gz

receiving incremental file list
hg38.fa.gz
  1,012,013,082 100%   38.31MB/s    0:00:25 (xfr#1, to-chk=0/1)

sent 43 bytes  received 1,011,705,923 bytes  35,498,454.95 bytes/sec
total size is 1,012,013,082  speedup is 1.00


In [ ]:
%cd /content/TREDNet_v1.1-colab
!pwd

/content/TREDNet_v1.1-colab
/content/TREDNet_v1.1-colab


In [4]:
# One-time: bring conda into Colab
!pip -q install condacolab
import condacolab; condacolab.install()   # runtime auto-restarts; re-run the next cells after

# Make conda fast: use libmamba solver
!conda install -n base -y conda-libmamba-solver
!conda config --set solver libmamba
!conda config --set channel_priority strict

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:11
🔁 Restarting kernel...
Channels:
 - conda-forge
Platform: linux-64
Solving environment: \ | / done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - conda-libmamba-solver


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2025.8.3   |       hbd8a1cb_0         151 KB  conda-forge
    certifi-2025.8.3           |     pyhd8ed1ab_0         155 KB  conda-forge
    conda-24.11.3              |  py311h38be061_0         1.1 MB  conda-forge
    openssl-3.5.3              |       h26f9b46_1         3.0 MB  conda-forge
    ------------------------------------------------------------
                                   

In [ ]:
!pwd

/content


In [1]:
# 2) Create the env from your file
!conda env create -f /content/TREDNet_v1.1-colab/tf2.15-gpu.yml

Channels:
 - conda-forge
 - bioconda
 - defaults
Platform: linux-64
Solving environment: \ | / - \ done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.3
    latest version: 25.7.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



cudnn-8.9.7.29       | 446.6 MB  | :   0% 0/1 [00:00<?, ?it/s]
libcublas-12.9.1.4   | 446.0 MB  | :   0% 0/1 [00:00<?, ?it/s]

tensorflow-base-2.15 | 321.9 MB  | :   0% 0/1 [00:00<?, ?it/s]


nccl-2.27.7.1        | 203.8 MB  | :   0% 0/1 [00:00<?, ?it/s]



libcusparse-12.5.10. | 199.2 MB  | :   0% 0/1 [00:00<?, ?it/s]




libcusolver-11.7.5.8 | 195.7 MB  | :   0% 0/1 [00:00<?, ?it/s]





libcufft-11.4.1.4    | 154.6 MB  | :   0% 0/1 [00:00<?, ?it/s]






cuda-nvrtc-12.9.86   | 64.1 MB   | :   0% 0/1 [00:00<?, ?it/s]







libcurand-10.3.10.19 | 44.0 MB   | :   0% 0/1 [00:00<?, ?it/s]








libnvjitlink-12.9.86 | 29.1 MB   | :   0% 0/1 [00:00<?, ?it/s]









cuda-nvcc-tools-12.

#Run TREDNet on GPU
Prerequisites:
* hg38 genome downloaded into the /fasta directory
* the following h5 files in /v0.1:
  * phase_one_model.h5
  * phase_one_weights.h5
  * phase_two_model.hdf5

In [ ]:
# Run TREDNet !!!
!sh /content/TREDNet_v1.1-colab/submit_testing.sh

Script directory (relative): /content/TREDNet_v1.1-colab
/content/TREDNet_v1.1-colab/input_training_trednet/testing
HNF4A_sample
running create_dataset
/content/TREDNet_v1.1-colab/input_training_trednet/testing/HNF4A_sample_positive_1kb.bed
/content/TREDNet_v1.1-colab/input_training_trednet/testing/HNF4A_sample_control_1kb.bed
/content/TREDNet_v1.1-colab/model_output/testing/HNF4A_sample.phase_two.hg38/phase_two_dataset.hdf5
running create_dataset_forphase2
running get phase 1 model
Generating the positive dataset
885
45
70
Generating the negative dataset
879
46
75
Merging positive and negative to single matrices

5/5 [==============================] - 5s 285ms/step

56/56 [==============================] - 2s 41ms/step

3/3 [==============================] - 1s 730ms/step
Saving to file: /content/TREDNet_v1.1-colab/model_output/testing/HNF4A_sample.phase_two.hg38/phase_two_dataset.hdf5
running train model()
Loading the dataset from: /content/TREDNet_v1.1-colab/model_output/testing/HNF

# Model Output Results

In [ ]:
# check outputs here:
%cd /content/TREDNet_v1.1-colab/model_output/testing/HNF4A_sample.phase_two.hg38/
!ls -alF


/content/TREDNet_v1.1-colab/model_output/testing/HNF4A_sample.phase_two.hg38
total 489124
drwxr-xr-x 2 root root      4096 Sep 23 16:23 ./
drwxr-xr-x 3 root root      4096 Sep 23 15:55 ../
-rw-r--r-- 1 root root        14 Sep 23 16:23 auc.txt
-rw-r--r-- 1 root root        53 Sep 23 16:23 fpr_threshold_scores.txt
-rw-r--r-- 1 root root  34030437 Sep 23 16:21 phase_two_dataset.hdf5
-rw-r--r-- 1 root root 116704608 Sep 23 16:21 phase_two_model.hdf5
-rw-r--r-- 1 root root 350074960 Sep 23 16:23 phase_two_weights.hdf5
-rw-r--r-- 1 root root        14 Sep 23 16:23 prc.txt
-rw-r--r-- 1 root root      3898 Sep 23 16:23 prc_values.txt
-rw-r--r-- 1 root root      1735 Sep 23 16:23 roc_values.txt
-rw-r--r-- 1 root root     12417 Sep 23 16:20 run_two_phase.HNF4A_sample.py


In [ ]:
# quick inspection model performance

!cat auc.txt
!cat prc.txt
!echo "FPR Threshold Scores"
!cat fpr_threshold_scores.txt

AUC: 0.787429
PRC: 0.750746
FPR Threshold Scores
10 	 0.511174
5 	 0.512385
3 	 0.512870
1 	 0.517448


# System Validation
The following can be used to troubleshoot a Colab GPU node environment for any sys config issues.

In [2]:
!nvidia-smi

Thu Sep 25 14:22:33 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   33C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
!conda info


     active environment : None
       user config file : /root/.condarc
 populated config files : /usr/local/.condarc
                          /root/.condarc
          conda version : 24.11.3
    conda-build version : not installed
         python version : 3.11.11.final.0
                 solver : libmamba (default)
       virtual packages : __archspec=1=broadwell
                          __conda=24.11.3=0
                          __cuda=12.4=0
                          __glibc=2.35=0
                          __linux=6.6.97=0
                          __unix=0=0
       base environment : /usr/local  (writable)
      conda av data dir : /usr/local/etc/conda
  conda av metadata url : None
           channel URLs : https://conda.anaconda.org/conda-forge/linux-64
                          https://conda.anaconda.org/conda-forge/noarch
          package cache : /usr/local/pkgs
                          /root/.conda/pkgs
       envs directories : /usr/local/envs
                        

In [4]:
!conda env list


# conda environments:
#
base                   /usr/local
tf2.15-gpu             /usr/local/envs/tf2.15-gpu



In [ ]:
import sys
print(sys.executable)

/usr/bin/python3.real


In [5]:
# 3) Use the env (no need to "activate" in notebooks)
!conda run -n tf2.15-gpu python -c "import tensorflow as tf; print('TF', tf.__version__)"


TF 2.15.0

2025-09-25 14:22:52.342320: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-09-25 14:22:52.342376: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-09-25 14:22:52.343617: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-25 14:22:52.354036: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.



In [ ]:
import tensorflow as tf;
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
!ls /usr/local/envs

tf2.15-gpu


In [ ]:
!ls -alF /usr/local/envs/tf2.15-gpu/bin/python* # (inside the tf2.15-gpu env)

lrwxrwxrwx 1 root root       10 Sep 23 15:26 /usr/local/envs/tf2.15-gpu/bin/python -> python3.10*
lrwxrwxrwx 1 root root       10 Sep 23 15:26 /usr/local/envs/tf2.15-gpu/bin/python3 -> python3.10*
lrwxrwxrwx 1 root root       10 Sep 23 15:26 /usr/local/envs/tf2.15-gpu/bin/python3.1 -> python3.10*
-rwxr-xr-x 1 root root 17292632 Sep 23 15:25 /usr/local/envs/tf2.15-gpu/bin/python3.10*
-rwxr-xr-x 1 root root     3531 Sep 23 15:25 /usr/local/envs/tf2.15-gpu/bin/python3.10-config*
lrwxrwxrwx 1 root root       17 Sep 23 15:26 /usr/local/envs/tf2.15-gpu/bin/python3-config -> python3.10-config*


In [8]:
!which python # (outside of the tf2.15-gpu env)

/usr/local/bin/python


In [12]:
!ls -alF /usr/local/bin/python* # ... and python points to 3.11

lrwxrwxrwx 1 root root       10 Sep 25 14:17 /usr/local/bin/python -> python3.11*
lrwxrwxrwx 1 root root       10 Sep 25 14:17 /usr/local/bin/python3 -> python3.11*
lrwxrwxrwx 1 root root       10 Sep 25 14:17 /usr/local/bin/python3.1 -> python3.11*
-rwxrwxr-x 1 root root 25381336 Sep 25 14:17 /usr/local/bin/python3.11*
-rwxrwxr-x 1 root root     3425 Sep 25 14:17 /usr/local/bin/python3.11-config*
lrwxrwxrwx 1 root root       17 Sep 25 14:17 /usr/local/bin/python3-config -> python3.11-config*


In [ ]:
!python --version

Python 3.11.11


In [ ]:
!conda run -n tf2.15-gpu python --version  # Notice TREDNet v1.1 env uses python 3.10

Python 3.10.18



In [ ]:
!ls /usr/local

bin		 cuda-12	   lib		sbin
colab		 cuda-12.5	   libexec	share
compiler_compat  dist_metrics.pxd  LICENSE.md	shell
_conda		 envs		   LICENSE.txt	src
condabin	 etc		   man		ssl
conda-meta	 games		   opt		x86_64-conda_cos6-linux-gnu
cuda		 include	   pkgs		x86_64-conda-linux-gnu


In [ ]:
!echo $PATH

/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin


In [ ]:
!python -c "import sys; print('\n'.join(sys.path))"


/env/python
/usr/local/lib/python311.zip
/usr/local/lib/python3.11
/usr/local/lib/python3.11/lib-dynload
/usr/local/lib/python3.11/site-packages


In [ ]:
# should see python3.10 installed in the tf2.15-gpu conda env
!conda run -n tf2.15-gpu python -c "import sys; print('\n'.join(sys.path))"


/env/python
/usr/local/envs/tf2.15-gpu/lib/python310.zip
/usr/local/envs/tf2.15-gpu/lib/python3.10
/usr/local/envs/tf2.15-gpu/lib/python3.10/lib-dynload
/usr/local/envs/tf2.15-gpu/lib/python3.10/site-packages



In [ ]:
# The following path is where all the extra packages needed by
# TREDNet are installed:
!ls /usr/local/envs/tf2.15-gpu/lib/python3.10/site-packages

absl
absl_py-2.3.1.dist-info
aiohappyeyeballs
aiohappyeyeballs-2.6.1.dist-info
aiohttp
aiohttp-3.12.15.dist-info
aiosignal
aiosignal-1.4.0.dist-info
asttokens
asttokens-3.0.0.dist-info
astunparse
astunparse-1.6.3.dist-info
async_timeout
async_timeout-5.0.1.dist-info
attr
attrs
attrs-25.3.0.dist-info
Bio
biopython-1.85.dist-info
BioSQL
bleach
bleach-6.2.0.dist-info
blinker
blinker-1.9.0.dist-info
bokeh
bokeh-3.7.3.dist-info
brotli-1.1.0.dist-info
_brotli.cpython-310-x86_64-linux-gnu.so
brotli.py
cached_property-1.5.2.dist-info
cached_property.py
cachetools
cachetools-5.5.2.dist-info
certifi
certifi-2025.8.3.dist-info
cffi
cffi-1.17.1.dist-info
_cffi_backend.cpython-310-x86_64-linux-gnu.so
charset_normalizer
charset_normalizer-3.4.3.dist-info
click
click-8.3.0.dist-info
colorama
colorama-0.4.6.dist-info
colorcet
colorcet-3.1.0.dist-info
comm
comm-0.2.3.dist-info
contourpy
contourpy-1.3.2.dist-info
cryptography
cryptography-45.0.7.dist-info
cycler
cycler-0.12.1.dist-info
dateutil
debugpy


In [ ]:
!mamba list

# packages in environment at /usr/local:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                 conda_forge    conda-forge
_openmp_mutex             4.5                       2_gnu    conda-forge
archspec                  0.2.3              pyhd8ed1ab_0    conda-forge
boltons                   24.0.0             pyhd8ed1ab_1    conda-forge
brotli-python             1.1.0           py311hfdbb021_2    conda-forge
bzip2                     1.0.8                h4bc722e_7    conda-forge
c-ares                    1.34.4               hb9d3cd8_0    conda-forge
ca-certificates           2025.8.3             hbd8a1cb_0    conda-forge
certifi                   2025.8.3           pyhd8ed1ab_0    conda-forge
cffi                      1.17.1          py311hf29c0ef_0    conda-forge
charset-normalizer        3.4.1              pyhd8ed1ab_0    conda-forge
colorama                  0.4.6              pyhd8ed1ab_1    conda-forge
conda         

In [ ]:
# should report /env/python
!echo $PYTHONPATH

/env/python


In [ ]:
!ls /usr/local/lib/

cmake				  libmkl_vml_avx512.so.2
itcl4.2.3			  libmkl_vml_cmpt.so.2
julia				  libmkl_vml_def.so.2
krb5				  libmkl_vml_mc3.so.2
libarcher.so			  libncurses++.a
libarcher_static.a		  libncurses.so
libarchive.a			  libncurses.so.6
libarchive.so			  libncurses.so.6.5
libarchive.so.13		  libncurses++w.a
libarchive.so.13.7.7		  libncursesw.so
libatomic.so			  libncursesw.so.6
libatomic.so.1			  libncursesw.so.6.5
libatomic.so.1.2.0		  libnghttp2.so
libbz2.a			  libnghttp2.so.14
libbz2.so			  libnghttp2.so.14.28.3
libbz2.so.1.0			  libnsl.so
libbz2.so.1.0.8			  libnsl.so.3
libcares.so			  libnsl.so.3.0.0
libcares.so.2			  libomp-fallback-cassert.spv
libcares.so.2.19.3		  libomp-fallback-cmath-fp64.spv
libcharset.a			  libomp-fallback-cmath.spv
libcharset.so			  libomp-fallback-complex-fp64.spv
libcharset.so.1			  libomp-fallback-complex.spv
libcharset.so.1.0.0		  libomp-fallback-cstring.spv
libcom_err.so			  libomp-fallback-imf-fp64.spv
libcom_err.so.3			  libomp-fallback-imf.spv
libcom_er

In [6]:
!conda --version

conda 24.11.3


In [7]:
# !conda run -n tf2.15-gpu python sanity_check.py

import tensorflow as tf

# sanity check
print("TF:", tf.__version__)
print("Built with CUDA:", tf.test.is_built_with_cuda())
print("GPUs:", tf.config.list_physical_devices('GPU'))

#confirm cuda libs loaded
from ctypes import cdll

# If these fail, your runtime isn't seeing the right libs
cuda_libs = [
    "libcudart.so",
    "libcublas.so",
    "libcublasLt.so",
    "libcufft.so",
    "libcurand.so",
    "libcusolver.so",
    "libcusparse.so",
    "libnvinfer.so",
    "libnvinfer_plugin.so",
    "libnvjpeg.so",
    "libnvrtc.so",
    "libnvToolsExt.so",
    "libcudnn.so", # newer versions might not have the .8 suffix
    "libcudnn.so.8", # some older versions might still have .8
]

for lib in cuda_libs:
    try:
        cdll.LoadLibrary(lib)
        print("Loaded", lib)
    except OSError as e:
        print("Missing", lib, e)

TF: 2.19.0
Built with CUDA: True
GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Loaded libcudart.so
Loaded libcublas.so
Loaded libcublasLt.so
Loaded libcufft.so
Loaded libcurand.so
Loaded libcusolver.so
Loaded libcusparse.so
Missing libnvinfer.so libnvinfer.so: cannot open shared object file: No such file or directory
Missing libnvinfer_plugin.so libnvinfer_plugin.so: cannot open shared object file: No such file or directory
Loaded libnvjpeg.so
Loaded libnvrtc.so
Loaded libnvToolsExt.so
Loaded libcudnn.so
Missing libcudnn.so.8 libcudnn.so.8: cannot open shared object file: No such file or directory


Since `libcudnn.so` was successfully loaded, the warning about `libcudnn.so.8` is just indicating that a specific older version of the library was not found. As long as the correct version (libcudnn.so) is loaded and recognized by TensorFlow (which it appears to be, given the `Built with CUDA: True` and the listed GPU), TREDNet is able to utilize the GPU.

In [ ]:
#!conda run -n tf2.15-gpu python sanity_check.py

In [ ]:
!nvidia-smi
#!ls /opt/nvidia/

Tue Sep 23 15:50:50 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pwd
!ls
%cd TREDNet_v1.1-colab
!pwd

/content/TREDNet_v1.1-colab
back_run_two_phase.py	list_of_biosamples_testing.txt	tf2.15-gpu.yml
condacolab_install.log	README.md			TREDNet.ipynb
fasta			submit_testing.sh		v0.1
input_training_trednet	test.py
[Errno 2] No such file or directory: 'TREDNet_v1.1-colab'
/content/TREDNet_v1.1-colab
/content/TREDNet_v1.1-colab


In [ ]:
import condacolab
condacolab.install()  # this will restart the runtime

✨🍰✨ Everything looks OK!


In [ ]:
import sys; print(sys.version)

3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]


# End of System Validation Tools
